In [41]:
import os
import wave
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Set the path of the directory containing the WAV files
directory_path = 'audio'

# Initialize lists to store audio data and labels
audio_data_list = []
labels_list = []

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    # Check if the file is a WAV file
    if filename.endswith('.wav'):
        # Open the WAV file
        file_path = os.path.join(directory_path, filename)
        audio_file = wave.open(file_path, 'rb')
        
        # Get the sample rate
        sample_rate = audio_file.getframerate()
        
        # Read all frames from the file
        audio_frames = audio_file.readframes(audio_file.getnframes())
        
        # Close the file
        audio_file.close()
        
        # Convert the frames to a NumPy array
        audio_data = np.frombuffer(audio_frames, dtype=np.int16)
        
        # Append the audio data and label to the lists
        audio_data_list.append(audio_data)
        labels_list.append(filename.split('_')[0])  # Assumes that the label is the first part of the filename
        print(f'Loaded file {filename} with sample rate {sample_rate} and length {len(audio_data)}')
# Convert the lists to NumPy arrays
audio_data_array = np.array(audio_data_list, dtype=object)
labels_array = np.array(labels_list)

# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(audio_data_array, labels_array, test_size=0.2, random_state=42)




Loaded file 84_121123_000009_000000.wav with sample rate 24000 and length 87840
Loaded file 84_121123_000010_000000.wav with sample rate 24000 and length 145440
Loaded file test1.wav with sample rate 24000 and length 42481
Loaded file 84_121123_000009_000007.wav with sample rate 24000 and length 152880
Loaded file 84_121123_000008_000004.wav with sample rate 24000 and length 129120
Loaded file 84_121123_000008_000003.wav with sample rate 24000 and length 396000
Loaded file 84_121123_000008_000002.wav with sample rate 24000 and length 81120
Loaded file 84_121123_000008_000000.wav with sample rate 24000 and length 100800
Loaded file 84_121123_000008_000001.wav with sample rate 24000 and length 132720
Loaded file 84_121123_000009_000008.wav with sample rate 24000 and length 33840
Loaded file 84_121123_000007_000001.wav with sample rate 24000 and length 24960


In [47]:
import os
import wave
import numpy as np
from sklearn.model_selection import train_test_split

class SVM:
    def __init__(self, learning_rate=1e-3, lambda_param=1e-2, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def _init_weights_bias(self, X):
        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        self.b = 0

    def _get_cls_map(self, y):
        # if y=0 then map to -1
        return np.where(y <= 0, -1, 1)

    def fit(self, X, y):
        # init weights & biases
        self._init_weights_bias(X)
        # map binary class to {-1, 1}
        self.cls_map = self._get_cls_map(y)
        
        for _ in range(self.n_iters):
            for idx, x in enumerate(X):
                # check if data point satisfies the constraint
                constrain = self._satisfy_constraint(x, idx)
                # compute the gradients accordingly
                dw, db = self._get_gradients(constrain, x, idx)
                # update weights & biases
                self._update_weights_bias(dw, db)

    def predict(self, X):
        linear_model = np.dot(X, self.w) + self.b
        return np.sign(linear_model)

    def _satisfy_constraint(self, x, idx):
        linear_model = np.dot(x, self.w) + self.b 
        return self.cls_map[idx] * linear_model >= 1
    
    def _get_gradients(self, constrain, x, idx):
        # if data point lies on the correct side
        if constrain:
            dw = self.lambda_param * self.w
            db = 0
            return dw, db
        # if data point is on the wrong side
        dw = self.lambda_param * self.w - np.dot(self.cls_map[idx], x)
        db = - self.cls_map[idx]
        return dw, db
    
    def _update_weights_bias(self, dw, db):
        self.w -= self.lr * dw
        self.b -= self.lr * db


# Set the path of the directory containing the WAV files
directory_path = 'audio'

# Initialize lists to store audio data and labels
audio_data_list = []
labels_list = []

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    # Check if the file is a WAV file
    if filename.endswith('.wav'):
        # Open the WAV file
        file_path = os.path.join(directory_path, filename)
        audio_file = wave.open(file_path, 'rb')
        
        # Get the sample rate
        sample_rate = audio_file.getframerate()
        
        # Read all frames from the file
        audio_frames = audio_file.readframes(audio_file.getnframes())
        
        # Close the file
        audio_file.close()
        
        # Convert the frames to a NumPy array
        audio_data = np.frombuffer(audio_frames, dtype=np.int16)
        
        # Append the audio data and label to the lists
        audio_data_list.append(audio_data)
        labels_list.append(filename.split('_')[0])  # Assumes that the label is the first part of the filename
        print(f'Loaded file {filename} with sample rate {sample_rate} and length {len(audio_data)}')


clf = SVM(n_iters=1000)

predictions = clf.predict(test_data)

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true==y_pred) / len(y_true)
    return accuracy

print("SVM Accuracy: ", accuracy(test_labels, predictions))

Loaded file 84_121123_000009_000000.wav with sample rate 24000 and length 87840
Loaded file 84_121123_000010_000000.wav with sample rate 24000 and length 145440
Loaded file test1.wav with sample rate 24000 and length 42481
Loaded file 84_121123_000009_000007.wav with sample rate 24000 and length 152880
Loaded file 84_121123_000008_000004.wav with sample rate 24000 and length 129120
Loaded file 84_121123_000008_000003.wav with sample rate 24000 and length 396000
Loaded file 84_121123_000008_000002.wav with sample rate 24000 and length 81120
Loaded file 84_121123_000008_000000.wav with sample rate 24000 and length 100800
Loaded file 84_121123_000008_000001.wav with sample rate 24000 and length 132720
Loaded file 84_121123_000009_000008.wav with sample rate 24000 and length 33840
Loaded file 84_121123_000007_000001.wav with sample rate 24000 and length 24960


TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'